<a href="https://colab.research.google.com/github/amun83/Koscom-Financial-Data-Bootcamp/blob/master/FD104_Oilprice_com_News_Text_Mining_and_One_More_Thing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping News Articles: https://oilprice.com/

In [0]:
ON_COLAB  = True
DATA_PATH = ''

In [0]:
!pip install -q --upgrade pandas_datareader
!pip install -q --upgrade finance-datareader
!pip install -q --upgrade html5lib
!pip install -q --upgrade lxml
!pip install -q --upgrade seaborn
!pip install -q --upgrade gspread

In [0]:
# utilities
import datetime
import re
import csv
from IPython.display import display

# web scraping
import requests
import urllib.parse
from bs4 import BeautifulSoup as bs
import pandas_datareader as pdr
import FinanceDataReader as fdr

# Math/Stat
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
#import math

# Machine Learning
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import LSTM

# visualization
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
%matplotlib inline

In [106]:
if ON_COLAB :
  from google.colab import drive
  drive.mount('/gdrive')
  DATA_PATH = '/gdrive/My Drive/sandbox/'

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
pattern = re.compile(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s(\d\d),\s(\d{4})')
# Example: By Julianne Geiger - Dec 14, 2018, 8:00 PM CST

In [0]:
def get_news_urls_by_page(page_num):
  page_url = 'https://oilprice.com/Latest-Energy-News/World-News/Page-' + str(page_num) + '.html'
  # https://oilprice.com/Latest-Energy-News/World-News/Page-865.html

  try:
    r = requests.get(page_url)
    print('Request(GET) >> OK: ' + page_url)
  except:
    print('Request(GET) >> Failed: ' + page_url)
    return None

  soup = bs(r.text)
  items = soup.find_all('div', {"class":"categoryArticle__content"})

  if not items:
    print('Request(GET) >> Empty Page: ' + page_url)
    return None
  
  return [item.find('a', href = True)['href'] for item in items]

In [0]:
def get_news_content(url):
  try:
    get_news_content.counter += 1
  except:
    get_news_content.counter = 0
  
  try:
    r = requests.get(url)
    print('[' + str(get_news_content.counter) + '] Request(GET) >> OK: ' + url)
  except:
    print('[' + str(get_news_content.counter) + '] Request(GET) >> Failed: ' + url)
    return None

  soup = bs(r.text)

  title = soup.find('h1').text

  date = soup.find('span', {'class':'article_byline'}).text

  match = re.search(pattern, date)
  mmm, dd, yyyy = match.groups(0)
  date = yyyy + "-" + mmm + "-" + dd

  raw_content = soup.find('div', {'id':'news-content'}).find_all('p')
  content = ' '.join([p.text.strip() for p in raw_content])
  
  return date, title, content

In [110]:
news_urls =[]

for i in range(1, 10):
  urls = get_news_urls_by_page(i)
  if not urls:
    break
    
  news_urls += urls

Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-1.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-2.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-3.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-4.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-5.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-6.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-7.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-8.html
Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Page-9.html


In [119]:
news_urls[-1]

'https://oilprice.com/Latest-Energy-News/World-News/US-Oil-Gas-Reserves-Hit-Record-Highs.html'

In [0]:
with open(DATA_PATH + 'oilprice.com_news_urls.csv', 'w') as f:
  fw = csv.writer(f)
  
  fw.writerows([[url] for url in news_urls])

In [117]:
news = [get_news_content(url) + (url.split('/')[-1],) for url in news_urls]

[90] Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/France-Extends-State-Guarantee-For-Total-Participated-Russian-LNG-Projects.html
[91] Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Saudi-Oil-Minister-Discusses-Further-Cooperation-With-Worlds-Largest-Refiner.html
[92] Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Pemex-Boosts-2019-Budget-14-With-Focus-On-Producing-Fields.html
[93] Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Environment-Conscious-Quebecois-Buy-More-Gas-Guzzlers.html
[94] Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Climate-Change-Conference-Strikes-Agreement-On-Rulebook.html
[95] Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Cuadrilla-Halts-Fracking-Again-Operations-As-Tremors-Persist.html
[96] Request(GET) >> OK: https://oilprice.com/Latest-Energy-News/World-News/Tesla-Cuts-Prices-In-China-As-Beijing-Suspends-New-US-Ca

In [118]:
news[-1]

('2018-Nov-29',
 'U.S. Oil, Gas Reserves Hit Record Highs',
 'Higher prices and continued shale resource development helped push U.S. crude oil and natural gas proved reserves to new record highs in 2017, the Energy Information Administration (EIA) said in a report on Thursday. Proved reserves of U.S. crude oil jumped by 19.5 percent in 2017 from the end of 2016, reaching 39.2 billion barrels and beating the previous record of 39.0 billion barrels set in 1970. Crude oil reserves reached an all-time in 2017, as higher prices typically raise reserve estimates because exploration and production companies believe that more of their resource bases can be produced. Last year, the average WTI Crude spot price rose by 20 percent compared to 2016, exceeding $60 a barrel for the first time since June 2015 and helping to drive increases in reserves, the EIA noted.  In its report for 2016, EIA said that the development in the Permian led to Texas showing the largest net increase in proved reserves

In [120]:
[a for a in news if len(a[2]) < 1024]

[]

In [0]:
with open(DATA_PATH + 'oilprice.com_news.csv', 'w') as f:
  fw = csv.writer(f)
  
  fw.writerows(news)